# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [125]:
nltk.download(['punkt','stopwords', 'wordnet', 'averaged_perceptron_tagger'])
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [147]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sqlalchemy import create_engine


In [127]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table('MessageDatabase', engine)  
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)


### 2. Write a tokenization function to process your text data

In [128]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [129]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [130]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [132]:
y_pred_names = pd.DataFrame(y_pred, columns = y_test.columns)
def display_results(y_test, y_pred):
    for column in y_test.columns:
        print('column_name: {}\n'.format(column))
        print(classification_report(y_test[column], y_pred_names[column]))

In [133]:
y_pred = pipeline.predict(X_test)
display_results(y_test, y_pred)

column_name: related

             precision    recall  f1-score   support

          0       0.21      0.18      0.20      1507
          1       0.76      0.79      0.78      4987
          2       0.00      0.00      0.00        51

avg / total       0.63      0.65      0.64      6545

column_name: request

             precision    recall  f1-score   support

          0       0.83      0.93      0.88      5450
          1       0.17      0.07      0.10      1095

avg / total       0.72      0.78      0.75      6545

column_name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6519
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6545

column_name: aid_related

             precision    recall  f1-score   support

          0       0.58      0.77      0.67      3854
          1       0.40      0.21      0.28      2691

avg / total       0.51      0.54      0.51      654

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [134]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_p

In [148]:
clf = AdaBoostClassifier()
pipeline_2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(clf))
])
parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0],
    'clf__estimator__n_estimators': [10, 20]
}


cv = GridSearchCV(pipeline_2, param_grid=parameters, cv=5, n_jobs=-1, verbose=3) 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
best_params = cv.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=20 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=10 
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estima

In [ ]:
#burayı henüz test etmedim.
print(classification_report(y_test.values, y_pred_3_test, target_names=y.columns.values))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.